In [1]:
import Generate as G
from voyage.Manager import VoyageManager
import Optimize as O
import pickle as pkl

from voyage.Ship.Ship import cShip
from voyage.Ship.Dynamics import Hull, Propeller, Rudder
from voyage.Ship.Env import Current, Wave, Wind
from voyage.Ship.Machinery import MainEngine
from voyage.Ship.Geo import Waterdepth
import json
from voyage.node_dict import coordlist
from Define import *

In [2]:
eng_data_dict = {
    "ShipPrincipal" : 'data/ShipPrincipal.json',
    "ShipSelfPropulsion" : 'data/ShipSelfPropulsion.json',
    "HullResistance" : 'data/HullResistance.json',
    "EnginePerformance" : 'data/EnginePerformance.json',
    "PropellerPOW" : 'data/PropellerPOW.json',
    "WindLoadCoefficient" : 'data/WindLoadCoefficient.json',
    "WaveDriftSurgeQTF" : 'data/WaveDriftSurgeQTF.json'
}
p_weather_pickle_path = 'data/weather.pickle'
p_depth_pickle_path = 'data/waterdepth.pickle'

p_graph_data = pkl.load(open("data/graph_23_300.pickle", 'rb'))

Options = dict()

In [3]:
def Make_ShipModel(p_weather_pickle_path:str, p_depth_pickle_path):
    #region load data
    # eng_data_dict = get_engineering_data(p_conn, p_ship_id)
    principal = json.load(open(eng_data_dict["ShipPrincipal"],'r'))
    propulsion =  json.load(open(eng_data_dict["ShipSelfPropulsion"], 'r'))
    hull_res =  json.load(open(eng_data_dict["HullResistance"], 'r'))
    main_engine =  json.load(open(eng_data_dict["EnginePerformance"], 'r'))
    propeller =  json.load(open(eng_data_dict["PropellerPOW"], 'r'))
    wind_coef =  json.load(open(eng_data_dict["WindLoadCoefficient"], 'r'))
    wave_qtf =  json.load(open(eng_data_dict["WaveDriftSurgeQTF"], 'r'))
    forecast = pkl.load(open(p_weather_pickle_path, 'rb'))
    #end region load data

    waterdepth = pkl.load(open(p_depth_pickle_path, 'rb'))

    #region set engineering data
    ship_principal = Data_ShipPrincipal
    ship_principal.Set_Data(principal)

    ship_propulsion = Data_ShipPropulsion()
    ship_propulsion.Set_Data(propulsion)

    ship_hull = Data_HullResistance()
    ship_hull.Set_Data(hull_res)
   
    ship_ME = Data_MainEngine()
    ship_ME.Set_Data(main_engine)

    ship_Propeller = Data_PropellerPOW()
    ship_Propeller.Set_Data(propeller)

    ship_windcoef = Data_WindLoadCoefficienct()
    ship_windcoef.Set_Data(wind_coef)

    ship_waveqtf = Data_WaveDriftSurgeQTF()
    ship_waveqtf.Set_Data(wave_qtf)

    ship_wind = Data_Forecast_Wind()
    ship_wave = Data_Forecast_Wave()
    ship_current = Data_Forecast_Current()
    ship_wind.Set_Data(forecast)
    ship_wave.Set_Data(forecast)
    ship_current.Set_Data(forecast)
    #end region set engineering data

    #region make ship model
    ship = cShip(ship_principal, ship_propulsion)
    hull = Hull.cHull(ship_hull)
    propeller1 = Propeller.cPropeller(ship_Propeller)
    propeller2 = Propeller.cPropeller(ship_Propeller)
    current = Current.cCurrent()
    current.setData(ship_current)
    wave = Wave.cWave(ship_wave, ship_waveqtf)
    wind = Wind.cWind()
    wind.setData(ship_wind, ship_windcoef)
    main_eng1 = MainEngine.cMainEngine(ship_ME)
    main_eng2 = MainEngine.cMainEngine(ship_ME)

    ship.setHull(hull)
    ship.setProp(propeller1)
    ship.setProp(propeller2)
    ship.setCurrent(current)
    ship.setWave(wave)
    ship.setWind(wind)
    ship.setME(main_eng1)
    ship.setME(main_eng2)
    #end region make ship model
    water_depth = Data_WaterDepth()
    water_depth.Set_Data(waterdepth)
    
    depth = Waterdepth.cWaterdepth(water_depth)
    ship.setWaterDepth(depth)
    return ship

In [4]:
def GeneratePlan(waypts_list, draft, p_graph_data:dict, p_weather_pickle_path:str, p_depth_pickle_path):
    RefPlan = G.Make_RefPlan(waypts_list, p_graph_data)

    Ship =Make_ShipModel(p_weather_pickle_path,p_depth_pickle_path)

    draft = float(draft)
    vm = VoyageManager(Ship)
    waypoints = vm.Generate(RefPlan.waypoints, draft)

    EvalPlan = Data_VoyagePlan()
    EvalPlan.Set_WP(waypoints)
    return {'waypoints' : EvalPlan.waypts_dict}


In [5]:
def OptimizePlan(waypts_list, draft, Options:dict, p_weather_pickle_path, p_depth_pickle_path):
    vp = Data_VoyagePlan()
    vp.Set_Data(waypts_list)
    Ship = Make_ShipModel(p_weather_pickle_path, p_depth_pickle_path)
    draft = float(draft)
    vm = VoyageManager(Ship)
    candid = dict()
    for idx, wp in enumerate(vp.waypoints):
        if idx == 0 or idx == len(vp.waypoints) : continue
        candid[float(wp.no)] = []
        for c in coordlist:
            c_lon = c[0]
            c_lat = c[1]
            if (float(wp.lon) - 2 <= c_lon <= float(wp.lon)+2) and (float(wp.lat) - 2 <= c_lat <= float(wp.lat) + 2):
                c_wp = Data_WayPoint()
                c_wp.lat = c_lat
                c_wp.lon = c_lon
                c_wp.no = wp.no
                candid[float(wp.no)].append(c_wp)
    Options = Data_OptimizeOption(Options)
    waypoints = vm.Optimize(vp.waypoints, candid, draft, Options)
    OptPlan = Data_VoyagePlan()
    OptPlan.Set_WP(waypoints)

    return  {'waypoints' : OptPlan.waypts_dict}


In [6]:
dep = [129.057636, 35.121357]
arr = [121.580736087328, 28.7094591823187]
time = ['2024-10-3 14:00:00', '2024-11-1 17:00:00']
waypts_list = [{'no': '1', 'code':'', 'lat': dep[1], 'lon': dep[0], 'time' : time[0]}, {'no':'2',  'code':'', "lat" : arr[1], 'lon' : arr[0], 'time' : time[1]}]
draft = 10.4


In [7]:
gen_wpts = GeneratePlan(waypts_list, draft, p_graph_data, p_weather_pickle_path,p_depth_pickle_path)

Reference route is done
[{'no': '1', 'code': '', 'time': '1727964000.0', 'lat': 35.121357, 'lon': 129.057636, 'dur': '-inf', 'sog': '-inf', 'sog_avg': '-inf', 'brg': '-inf', 'dist': '-inf', 'dist_total': '-inf', 'shaft_rpm': '-inf', 'equiv_mdo': '-inf', 'equiv_mdo_total': '-inf', 'winDir': '-inf', 'winSpd': '-inf', 'wavHgt': '-inf', 'wavDir': '-inf', 'curDir': '-inf', 'curSpd': '-inf'}, {'no': '1-1', 'code': '', 'time': '-inf', 'lat': 35.07739333, 'lon': 128.83029833, 'dur': '-inf', 'sog': '-inf', 'sog_avg': '-inf', 'brg': '-inf', 'dist': '-inf', 'dist_total': '-inf', 'shaft_rpm': '-inf', 'equiv_mdo': '-inf', 'equiv_mdo_total': '-inf', 'winDir': '-inf', 'winSpd': '-inf', 'wavHgt': '-inf', 'wavDir': '-inf', 'curDir': '-inf', 'curSpd': '-inf'}, {'no': '1-2', 'code': '', 'time': '-inf', 'lat': 34.46382833, 'lon': 128.17792667, 'dur': '-inf', 'sog': '-inf', 'sog_avg': '-inf', 'brg': '-inf', 'dist': '-inf', 'dist_total': '-inf', 'shaft_rpm': '-inf', 'equiv_mdo': '-inf', 'equiv_mdo_total': '

In [8]:
OptimizePlan(gen_wpts['waypoints'], draft, Options, p_weather_pickle_path, p_depth_pickle_path)

{'waypoints': [{'no': '1',
   'code': '',
   'time': '2024-10-03 14:00:00',
   'lat': 35.121357,
   'lon': 129.057636,
   'dur': 0.0,
   'sog': 4.44244696905944,
   'sog_avg': 0.0,
   'brg': 26.14,
   'dist': 0.0,
   'dist_total': 0.0,
   'shaft_rpm': 69.6978235244751,
   'equiv_mdo': 0.15782519350574015,
   'equiv_mdo_total': 0,
   'winDir': 218.38843399999988,
   'winSpd': 16.68155711111831,
   'wavHgt': 2.799999952316284,
   'wavDir': 185.8738619999999,
   'curDir': 241.0,
   'curSpd': 0.37572860740715197},
  {'no': '1-1',
   'code': '',
   'time': '2024-10-04 04:00:18',
   'lat': 36.92468667,
   'lon': 130.16747833,
   'dur': 14.005044314538942,
   'sog': 1.7602565588814436,
   'sog_avg': '-inf',
   'brg': -157.04,
   'dist': 223.98,
   'dist_total': 223.98,
   'shaft_rpm': 27.49591827392578,
   'equiv_mdo': 0.16890000000000002,
   'equiv_mdo_total': 0.0023808575334716203,
   'winDir': 183.0,
   'winSpd': 20.824686288530273,
   'wavHgt': 3.784937300681162,
   'wavDir': 184.0,
   'c

In [ ]:
import plotly.graph_objects as go

# 경도와 위도를 각각 분리
lons, lats = zip(*lonlats)

# 지도에 경로 표시
fig = go.Figure()

# 경로 추가 (선형 경로)
fig.add_trace(go.Scattermapbox(
    mode="markers+lines",
    lon=lons,
    lat=lats,
    line=dict(width=1, color='green'),  
    marker=dict(size=3, color='green')  
))

# 지도 스타일 및 설정
fig.update_layout(
    mapbox=dict(
        style="open-street-map",  # 지도 스타일: OpenStreetMap
        zoom=4,  # 지도 확대 수준
        center=dict(lat=sum(lats)/len(lats), lon=sum(lons)/len(lons))  # 경로의 중심으로 이동
    ),
    margin=dict(l=0, r=0, t=0, b=0)  # 여백 제거
)


